In [1]:
## imports
import pandas as pd
import numpy as np
import re
import requests
import yaml


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Example 1: no credentials; no wrapper

Site: National Assessment of Education Progress (NAEP)

Documentation: https://www.nationsreportcard.gov/api_documentation.aspx

Base link: https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx 

## 1.1 Query to pull some data

In [2]:
## using their example query of 2011 writing scores separated by gender
## based on here - https://stackoverflow.com/questions/40836749/pythonic-way-of-writing-a-single-line-long-string
## using the ( ) syntax to formulate a long
## string without linebreaks added
example_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')


example_naep_query


'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'

In [3]:
## use requests to call the api
naep_resp = requests.get(example_naep_query)
naep_resp
print(type(naep_resp))

## get the json contents of the response 
## here, we're assuming valid response
naep_resp_j = naep_resp.json()
naep_resp_j

## with result, turn it into a dataframe
naep_resp_d = pd.DataFrame(naep_resp_j['result'])
naep_resp_d

<Response [200]>

<class 'requests.models.Response'>


{'status': 200,
 'serviceVersion': '3.28.2025.1',
 'dwellTimeMS': '0',
 'avgWebHostCPUTotalLoad': 'N/A',
 'dataHitType': 'FROM_MEMORY',
 'Source': 'B11A',
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 139.099504632971,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 158.567104984955,
   'isStatDisp

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0


## 1.2 What happens if there's an error in our query?

In [4]:
## here's a query that from the documentation we know
## won't work since i modified year to 2025 which doesnt
## exist in the data
wrong_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025')

wrong_naep_query

'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025'

In [5]:
## use requests to call the api
naep_wrong_resp = requests.get(wrong_naep_query)
naep_wrong_resp

<Response [400]>

In [ ]:
## in the case of this particular api,
## the call returns some response but
## when we try to extract the json containing
## status or results, we get in an error
#naep_wrong_resp.json() # uncomment to see error

### 1.2.2 More all-purpose way of allowing remainder of calls to run: try, except

In [6]:
## putting it in a try; except as general error catching
try:
    results = naep_wrong_resp.json()['result']
except Exception as e:
    print('Failed to get result from API due to error:')
    print(e) # or just: pass

Failed to get result from API due to error:
Invalid control character at: line 1 column 293 (char 292)


### 1.2.3 Can usually also find more targeted way but that varies more across APIs

In [7]:
## if we wanted do more specific error catching,
## see that the status == 400 actually appears here
## so could write if else along those lines
naep_wrong_resp.text
naep_resp.text

if "System.Exception" in naep_wrong_resp.text:
    print("NAEP results not found")

'{"statusCode":400,"result": "System.Exception: The query \'SELECT DISTINCT Framework FROM Cycles WHERE Subject=\'WRI\' AND Cohort=2 AND CONVERT(VARCHAR(10),Year)+Sample IN (\'2025R3\')\' did not return exactly 1 framework. Make sure you can trend the years defined for the given subject and cohort.\r\n   at NRCDataService3.GetAdhocData.GetFramework(NDEContext& ndeContext, String subjectCode, List`1 yearSamples, String cohort) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2547\r\n   at NRCDataService3.GetAdhocData.PopulateBaseOrchestratorRequest() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2168\r\n   at NRCDataService3.GetAdhocData.ConstructRequest_Datapoint() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 807\r\n   at NRCDataService3.GetAdhocData.Page_Load(Object sender, EventArgs e) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 261"}'

'{"status":200,"serviceVersion":"3.28.2025.1","dwellTimeMS":"0","avgWebHostCPUTotalLoad":"N/A","dataHitType":"FROM_MEMORY","Source":"B11A","result": [{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"1","varValueLabel":"Male","value":139.099504632971,"isStatDisplayable":1,"errorFlag":0},{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"2","varValueLabel":"Female","value":158.567104984955,"isStatDisplayable":1,"errorFlag":0}]}'

NAEP results not found


## Activity 1: writing a function to make multiple, sequential calls

- Say we want to pull the data for grades 4, 8, and 12
- How can we write a function that iterates over a list of those grades and pulls the data for each grade?

**Note**: an ideal function would have arguments for each parameter in the API like subject, subscale, etc. Here we can leave those other parts constant

In [16]:
# your code here
def each_grade(grades):
    results = []
    for grade in grades:
        example_naep_query = (
            'https://www.nationsreportcard.gov/'
            'Dataservice/GetAdhocData.aspx?'
            f'type=data&subject=writing&grade={grade}&'
            'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')
        naep_resp = requests.get(example_naep_query)
        try:
            naep_resp_j = naep_resp.json()
            naep_resp_d = pd.DataFrame(naep_resp_j['result'])
            results.append(naep_resp_d)
        except Exception as e:
            print(e)
    return pd.concat(results)
grades = ['4', '8', '12']
new_data = each_grade(grades)
print(new_data.head())


Invalid control character at: line 1 column 293 (char 292)
   year sample yearSampleLabel  Cohort CohortLabel stattype subject  grade  \
0  2011     R3            2011       2     Grade 8    MN:MN     WRI      8   
1  2011     R3            2011       2     Grade 8    MN:MN     WRI      8   
0  2011     R3            2011       3    Grade 12    MN:MN     WRI     12   
1  2011     R3            2011       3    Grade 12    MN:MN     WRI     12   

   scale jurisdiction       jurisLabel variable variableLabel varValue  \
0  WRIRP           NP  National public   GENDER        Gender        1   
1  WRIRP           NP  National public   GENDER        Gender        2   
0  WRIRP           NP  National public   GENDER        Gender        1   
1  WRIRP           NP  National public   GENDER        Gender        2   

  varValueLabel       value  isStatDisplayable  errorFlag  
0          Male  139.099505                  1          0  
1        Female  158.567105                  1          0  

# 2. Example 2: needs credentials; no wrapper

Create an account here: https://www.yelp.com/developers/v3/manage_app

In [17]:
## get the key
API_KEY = "rAZn1RX6Tr2o999Ly8EvD6ofsNGBZg3CMDsDxAwKGTEdWIi1zm9g23h_1ll5oix2Z0hSDiRloteTewU35nBqydGvoq0LngcGlZPP6LERVywiS_uc18KMmtvg9qQuZXYx"

In [18]:
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Hanover,NH,03755"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()


<Response [200]>

In [19]:
## example business
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

{'id': 'wyV_NfYn4ZOfp_sHMDPcAw',
 'alias': 'bistro-at-six-hanover',
 'name': 'Bistro at Six',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/i4jvssUsxa79VYVLOD3TmQ/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/bistro-at-six-hanover?adjust_creative=ZmitoFfF-pb6UbUwNTC3_A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=ZmitoFfF-pb6UbUwNTC3_A',
 'review_count': 2,
 'categories': [{'alias': 'lounges', 'title': 'Lounges'},
  {'alias': 'cocktailbars', 'title': 'Cocktail Bars'},
  {'alias': 'tradamerican', 'title': 'American'}],
 'rating': 4.0,
 'coordinates': {'latitude': 43.7001146, 'longitude': -72.2877078},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '6 E South St',
  'address2': None,
  'address3': '',
  'city': 'Hanover',
  'zip_code': '03755',
  'country': 'US',
  'state': 'NH',
  'display_address': ['6 E South St', 'Hanover, NH 03755']},
 'phone': '+16036430600',
 'display_phone': '(603) 643-0600',
 'distance': 198.6517

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,wyV_NfYn4ZOfp_sHMDPcAw,bistro-at-six-hanover,Bistro at Six,https://s3-media2.fl.yelpcdn.com/bphoto/i4jvss...,False,https://www.yelp.com/biz/bistro-at-six-hanover...,2,"[{'alias': 'lounges', 'title': 'Lounges'}, {'a...",4.0,"{'latitude': 43.7001146, 'longitude': -72.2877...",[],$$,"{'address1': '6 E South St', 'address2': None,...",+16036430600,(603) 643-0600,198.651788
1,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,False,https://www.yelp.com/biz/base-camp-cafe-hanove...,256,"[{'alias': 'himalayan', 'title': 'Himalayan/Ne...",4.4,"{'latitude': 43.700626, 'longitude': -72.2887803}",[delivery],$$,"{'address1': '3 Lebanon St', 'address2': 'Ste ...",+16036432007,(603) 643-2007,196.139758
2,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA...,False,https://www.yelp.com/biz/mollys-restaurant-and...,561,"[{'alias': 'tradamerican', 'title': 'American'...",3.9,"{'latitude': 43.701144, 'longitude': -72.2894249}",[delivery],$$,"{'address1': '43 South Main St', 'address2': '...",+16036432570,(603) 643-2570,250.830160
3,KA8yhrd-ClVYMyOefXdVYg,lous-restaurant-and-bakery-hanover,Lou's Restaurant & Bakery,https://s3-media3.fl.yelpcdn.com/bphoto/VAx8H9...,False,https://www.yelp.com/biz/lous-restaurant-and-b...,430,"[{'alias': 'tradamerican', 'title': 'American'...",4.2,"{'latitude': 43.70143, 'longitude': -72.289001}",[delivery],$$,"{'address1': '30 S Main St', 'address2': '', '...",+16036433321,(603) 643-3321,245.079232
4,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,False,https://www.yelp.com/biz/sawtooth-kitchen-hano...,30,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",4.2,"{'latitude': 43.70158, 'longitude': -72.289641}",[],NaN,"{'address1': '33 S Main St', 'address2': '', '...",+16036435134,(603) 643-5134,242.607552


In [20]:
## more data-specific way of summarizing
## we're doing a simple approach and just retaining
## cols that have a simple str structure
## if doing for real, would want to extract things
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)


,id,alias,name,image_url,url,price,phone,display_phone
wyV_NfYn4ZOfp_sHMDPcAw,wyV_NfYn4ZOfp_sHMDPcAw,bistro-at-six-hanover,Bistro at Six,https://s3-media2.fl.yelpcdn.com/bphoto/i4jvss...,https://www.yelp.com/biz/bistro-at-six-hanover...,$$,+16036430600,(603) 643-0600
XVGEEIH5rVB2QzW-qywcJw,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,https://www.yelp.com/biz/base-camp-cafe-hanove...,$$,+16036432007,(603) 643-2007
8ybF6YyRldtZmU9jil4xlg,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA...,https://www.yelp.com/biz/mollys-restaurant-and...,$$,+16036432570,(603) 643-2570
KA8yhrd-ClVYMyOefXdVYg,KA8yhrd-ClVYMyOefXdVYg,lous-restaurant-and-bakery-hanover,Lou's Restaurant & Bakery,https://s3-media3.fl.yelpcdn.com/bphoto/VAx8H9...,https://www.yelp.com/biz/lous-restaurant-and-b...,$$,+16036433321,(603) 643-3321
5WW4g_LRwau29KyjZGLyAA,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,https://www.yelp.com/biz/sawtooth-kitchen-hano...,NaN,+16036435134,(603) 643-5134
neBEWvgHNhsXIBImCYx_6A,neBEWvgHNhsXIBImCYx_6A,murphy-s-on-the-green-hanover,Murphy’s on the Green,https://s3-media2.fl.yelpcdn.com/bphoto/76DCQr...,https://www.yelp.com/biz/murphy-s-on-the-green...,$$,+16036434075,(603) 643-4075
vMyN7JL5cJExJORgIobbQg,vMyN7JL5cJExJORgIobbQg,tuk-tuk-thai-cuisine-hanover,Tuk Tuk Thai Cuisine,https://s3-media4.fl.yelpcdn.com/bphoto/--bUG3...,https://www.yelp.com/biz/tuk-tuk-thai-cuisine-...,$$,+16032779192,(603) 277-9192


# Activity 2: pull restaurants in a different location

- Try running a business search query for your hometown or another place by constructing a query similar to `yelp_genquery` but changing the location parameter
- Other endpoints require feeding what's called the business' fusion id into the API. Take an id from `yelp_stronly.id` and use the documentation here to pull the reviews for that business: https://docs.developer.yelp.com/reference/v3_business_reviews
- **Challenge**: generalize the previous step by writing a function that (1) takes a list of business ids as an input, (2) calls the reviews API for each id, (3) returns the results, and (4) rowbinds all results, i.e. turns them into a single, usable DataFrame

In [26]:
# your code here
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Princeton,MA,01541"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)
yelp_genquery

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()

## example business
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

## more data-specific way of summarizing
## we're doing a simple approach and just retaining
## cols that have a simple str structure
## if doing for real, would want to extract things
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)




url = "https://api.yelp.com/v3/businesses/Ecdu5qYM09F647Uoez99vg/reviews?"
yelp_genresp2 = requests.get(url, headers = header)
yelp_genjson2 = yelp_genresp2.json()

## example business
yelp_genjson2['reviews'][0]

'https://api.yelp.com/v3/businesses/search?term=restaurants&location=Princeton,MA,01541'

<Response [200]>

{'id': 'OrcSHHcCX-W2U7IdjeTKjg',
 'alias': 'logan-s-table-leominster',
 'name': 'Logan’s Table',
 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/Ygt2VP8sqs8MnmhuAB-__A/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/logan-s-table-leominster?adjust_creative=ZmitoFfF-pb6UbUwNTC3_A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=ZmitoFfF-pb6UbUwNTC3_A',
 'review_count': 119,
 'categories': [{'alias': 'newamerican', 'title': 'New American'},
  {'alias': 'pizza', 'title': 'Pizza'}],
 'rating': 4.5,
 'coordinates': {'latitude': 42.52477, 'longitude': -71.75919},
 'transactions': [],
 'location': {'address1': '16 Central St',
  'address2': '',
  'address3': None,
  'city': 'Leominster',
  'zip_code': '01453',
  'country': 'US',
  'state': 'MA',
  'display_address': ['16 Central St', 'Leominster, MA 01453']},
 'phone': '+19784012962',
 'display_phone': '(978) 401-2962',
 'distance': 13038.248851929422}

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,OrcSHHcCX-W2U7IdjeTKjg,logan-s-table-leominster,Logan’s Table,https://s3-media4.fl.yelpcdn.com/bphoto/Ygt2VP...,False,https://www.yelp.com/biz/logan-s-table-leomins...,119,"[{'alias': 'newamerican', 'title': 'New Americ...",4.5,"{'latitude': 42.52477, 'longitude': -71.75919}",[],"{'address1': '16 Central St', 'address2': '', ...",+19784012962,(978) 401-2962,13038.248852,NaN
1,1wV4EkDtwbqWP-OndU8szg,annas-table-leominster,Anna's Table,https://s3-media1.fl.yelpcdn.com/bphoto/Gc1RMM...,False,https://www.yelp.com/biz/annas-table-leominste...,39,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.4,"{'latitude': 42.50158541689027, 'longitude': -...",[],"{'address1': '16 Brooks Pond Rd', 'address2': ...",+19788402662,(978) 840-2662,12831.168137,NaN
2,F1K8iHcH22xWJoLHJ898sw,mountainside-bakery-and-cafe-princeton-2,Mountainside Bakery & Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/R5jaiv...,False,https://www.yelp.com/biz/mountainside-bakery-a...,97,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.5,"{'latitude': 42.4760481898851, 'longitude': -7...",[],"{'address1': '213 Mountain Rd', 'address2': No...",+19784640300,(978) 464-0300,2117.423341,$$
3,nfeb9YXVrPx1cJOlgOHDNA,the-1761-old-mill-westminster,The 1761 Old Mill,https://s3-media2.fl.yelpcdn.com/bphoto/69Gl7g...,False,https://www.yelp.com/biz/the-1761-old-mill-wes...,310,"[{'alias': 'tradamerican', 'title': 'American'...",3.6,"{'latitude': 42.54922215480225, 'longitude': -...",[],"{'address1': '69 State Rd E', 'address2': None...",+19788745941,(978) 874-5941,10217.505680,$$
4,CFccQR7Fys9NkmeoWLid2Q,darios-ristorante-fitchburg,Dario's Ristorante,https://s3-media2.fl.yelpcdn.com/bphoto/pBYPCH...,False,https://www.yelp.com/biz/darios-ristorante-fit...,95,"[{'alias': 'italian', 'title': 'Italian'}]",4.6,"{'latitude': 42.584026, 'longitude': -71.802358}","[delivery, pickup]","{'address1': '655 Main St', 'address2': '', 'a...",+19786273974,(978) 627-3974,15779.503681,$$


,id,alias,name,image_url,url,phone,display_phone,price
OrcSHHcCX-W2U7IdjeTKjg,OrcSHHcCX-W2U7IdjeTKjg,logan-s-table-leominster,Logan’s Table,https://s3-media4.fl.yelpcdn.com/bphoto/Ygt2VP...,https://www.yelp.com/biz/logan-s-table-leomins...,+19784012962,(978) 401-2962,NaN
1wV4EkDtwbqWP-OndU8szg,1wV4EkDtwbqWP-OndU8szg,annas-table-leominster,Anna's Table,https://s3-media1.fl.yelpcdn.com/bphoto/Gc1RMM...,https://www.yelp.com/biz/annas-table-leominste...,+19788402662,(978) 840-2662,NaN
F1K8iHcH22xWJoLHJ898sw,F1K8iHcH22xWJoLHJ898sw,mountainside-bakery-and-cafe-princeton-2,Mountainside Bakery & Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/R5jaiv...,https://www.yelp.com/biz/mountainside-bakery-a...,+19784640300,(978) 464-0300,$$
nfeb9YXVrPx1cJOlgOHDNA,nfeb9YXVrPx1cJOlgOHDNA,the-1761-old-mill-westminster,The 1761 Old Mill,https://s3-media2.fl.yelpcdn.com/bphoto/69Gl7g...,https://www.yelp.com/biz/the-1761-old-mill-wes...,+19788745941,(978) 874-5941,$$
CFccQR7Fys9NkmeoWLid2Q,CFccQR7Fys9NkmeoWLid2Q,darios-ristorante-fitchburg,Dario's Ristorante,https://s3-media2.fl.yelpcdn.com/bphoto/pBYPCH...,https://www.yelp.com/biz/darios-ristorante-fit...,+19786273974,(978) 627-3974,$$
tkD81L69HUt0ZUzvdSYNog,tkD81L69HUt0ZUzvdSYNog,monti-farms-and-deli-princeton,Monti Farms & Deli,https://s3-media3.fl.yelpcdn.com/bphoto/9MD6Jn...,https://www.yelp.com/biz/monti-farms-and-deli-...,+19194284758,(919) 428-4758,NaN
xd-G4ijhAy3m3UUwecSF8w,xd-G4ijhAy3m3UUwecSF8w,albatross-westminster,Albatross,https://s3-media2.fl.yelpcdn.com/bphoto/zOECZh...,https://www.yelp.com/biz/albatross-westminster...,+19788745900,(978) 874-5900,NaN


{'id': 'WteK0vOSbkup7fz4Ov06dg',
 'url': 'https://www.yelp.com/biz/ramuntos-brick-and-brew-pizzeria-hanover?adjust_creative=ZmitoFfF-pb6UbUwNTC3_A&hrid=WteK0vOSbkup7fz4Ov06dg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=ZmitoFfF-pb6UbUwNTC3_A',
 'text': 'Exceeded exceptions. Good veg choices. Counter staff was happy to answer questions. Low key vibe when we visited. I image it can also get busy!',
 'rating': 5,
 'time_created': '2025-02-10 13:46:14',
 'user': {'id': 'YdPx900lH1RUMolBP7mbFA',
  'profile_url': 'https://www.yelp.com/user_details?userid=YdPx900lH1RUMolBP7mbFA',
  'image_url': 'https://s3-media4.fl.yelpcdn.com/photo/rySs1i_jdrapYZjWzf2JAA/o.jpg',
  'name': 'Timothy S.'}}